# Challenge 1

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def is_running_on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ and os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Interactive"
data_path = '/kaggle/input/' if is_running_on_kaggle() else 'data/'

#for dirname, _, filenames in os.walk(data_path):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [6]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

# Imports
import warnings
import matplotlib.pyplot as plt
from torchvision.io import read_image

class DatasetLoader(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [13]:
annotations_file = data_path + 'train.csv'
img_dir = data_path + 'train/train/'
training_data = DatasetLoader(annotations_file, img_dir)

print(training_data[1][0].shape)
print(training_data[1])

torch.Size([3, 32, 32])
(tensor([[[118, 108, 104,  ..., 158, 138, 123],
         [133, 123, 117,  ..., 135, 119, 116],
         [150, 125, 107,  ...,  99, 103, 125],
         ...,
         [205, 212, 225,  ..., 185, 208, 245],
         [111, 120, 129,  ..., 113, 142, 185],
         [ 93,  67,  60,  ..., 104,  97, 126]],

        [[109,  99,  95,  ..., 132, 112,  97],
         [124, 114, 108,  ..., 109,  93,  90],
         [141, 116,  98,  ...,  73,  77,  99],
         ...,
         [190, 197, 213,  ..., 173, 195, 232],
         [ 96, 105, 117,  ..., 100, 126, 169],
         [ 78,  52,  48,  ...,  91,  81, 110]],

        [[114, 104, 100,  ..., 143, 123, 108],
         [129, 119, 113,  ..., 120, 104, 101],
         [144, 119, 103,  ...,  86,  90, 112],
         ...,
         [195, 202, 217,  ..., 183, 205, 242],
         [101, 110, 121,  ..., 110, 137, 180],
         [ 83,  57,  52,  ..., 101,  92, 121]]], dtype=torch.uint8), 1)


In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TinyModel(nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Define fully connected layers
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # 32x32 image downscaled by 2x2 max pooling twice
        self.fc2 = nn.Linear(128, 1)  # Output layer with a single neuron for binary classification

    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # Flatten the output for fully connected layers
        x = x.view(-1, 32 * 8 * 8)
        # Fully connected layers with ReLU activation
        x = F.relu(self.fc1(x))
        # Output layer with sigmoid activation for binary classification
        x = torch.sigmoid(self.fc2(x))
        # x = x > 0.5
        return x

tinymodel = TinyModel()
BATCH_SIZE = 32
LEARNING_RATE = 1e-3

print('The model: ', tinymodel)

# Define the loss function
loss_fn = torch.nn.BCELoss()

# Define the optimizer
optimizer = torch.optim.SGD(tinymodel.parameters(), lr=LEARNING_RATE)

# Load the data
train_dataloader = torch.utils.data.DataLoader(training_data, BATCH_SIZE, shuffle=True)

# Train the model
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.float())
        print(pred)
        print('Interval: ', pred.min(), pred.max())
        y = y.reshape(-1, 1).float()
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, tinymodel, loss_fn, optimizer)

The model:  TinyModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)
Epoch 1
-------------------------------
tensor([[0.9531],
        [0.9862],
        [0.9632],
        [0.9191],
        [0.9906],
        [0.9915],
        [0.9497],
        [0.9284],
        [0.7989],
        [0.9436],
        [0.9340],
        [0.9143],
        [0.9705],
        [0.9940],
        [0.9527],
        [0.9930],
        [0.8560],
        [0.9673],
        [0.9557],
        [0.7863],
        [0.9592],
        [0.9906],
        [0.8588],
        [0.8278],
        [0.9840],
        [0.9334],
        [0.9675],
        [0.9739],
        [0.9713],
        [0.9819],
        [0.9512],
        [0.9792]], grad_fn=<SigmoidBackward0>)
Interval:  tensor(0.7863, grad_fn=<MinBackward1>) tens